In [3]:
#线程和进程
#单线程
from random import randint
from time import time, sleep


def download_task(filename):
    print('开始下载%s...' % filename)
    time_to_download = randint(1, 6)
    sleep(time_to_download)
    print('%s下载完成! 耗费了%d秒' % (filename, time_to_download))


def main():
    start = time()
    download_task('Python从入门到住院.pdf')
    download_task('Peking Hot.avi')
    end = time()
    print('总共耗费了%.2f秒.' % (end - start))


if __name__ == '__main__':
    main()

开始下载Python从入门到住院.pdf...
Python从入门到住院.pdf下载完成! 耗费了10秒
开始下载Peking Hot.avi...
Peking Hot.avi下载完成! 耗费了8秒
总共耗费了18.00秒.


In [5]:
# 多线程
# 通过process方法实现多线程。Windows下不支持
from multiprocessing import Process
from os import getpid
from random import randint
from time import time, sleep


def download_task(filename):
    print('启动下载进程，进程号[%d].' % getpid())
    print('开始下载%s...' % filename)
    time_to_download = randint(5, 10)
    sleep(time_to_download)
    print('%s下载完成! 耗费了%d秒' % (filename, time_to_download))


def main():
    start = time()
    p1 = Process(target=download_task, args=('Python从入门到住院.pdf', ))
    p1.start()
    p2 = Process(target=download_task, args=('Peking Hot.avi', ))
    p2.start()
    p1.join()
    p2.join()
    end = time()
    print('总共耗费了%.2f秒.' % (end - start))


if __name__ == '__main__':
    main()

总共耗费了0.11秒.


In [7]:
# 线程的执行次数
# 下面的函数并不能控制两个线程合计执行10次。
# 要解决这个问题比较简单的办法是使用multiprocessing模块中的Queue类
from multiprocessing import Process
from time import sleep

counter = 0


def sub_task(string):
    global counter
    while counter < 10:
        print(string, end='', flush=True)
        counter += 1
        sleep(0.01)

        
def main():
    Process(target=sub_task, args=('Ping', )).start()
    Process(target=sub_task, args=('Pong', )).start()


if __name__ == '__main__':
    main()

In [9]:
#多线程
# 通过Thread 实现多线程
from random import randint
from threading import Thread
from time import time, sleep


def download(filename):
    print('Begin to download %s' % filename)
    elaps_time=randint(3,8)
    sleep(elaps_time)
    print('download completed,has consumed %d seconds' % elaps_time)


def main():
    start=time()
    #start = time()
    t1 = Thread(target=download, args=('Python从入门到住院.pdf',))
    t1.start()
    t2 = Thread(target=download, args=('Peking Hot.avi',))
    t2.start()
    t1.join()
    t2.join()
    end = time()
    print('总共耗费了%.3f秒' % (end - start))


if __name__ == '__main__':
    main()

Begin to download Python从入门到住院.pdf
Begin to download Peking Hot.avi
download completed,occupy 3 seconds
download completed,occupy 6 seconds
总共耗费了6.004秒


In [12]:
#多线程
# 通过 继承 Thread 类下载
from random  import randint
from threading import Thread
import time as tm
class DownloadTask(Thread):
    def __init__(self,filename):
        super().__init__()
        self._filename=filename
    def run(self):
        print('Begin to download %s' % self._filename)
        elaps_time=randint(3,8)
        sleep(elaps_time)
        print('%s download completed,has consumed %d seconds' % (self._filename,elaps_time))
def main():
    start=time()
    t1=DownloadTask('Python从入门到住院.pdf')
    t2=DownloadTask('how to pass a interview.pdf')
    t1.start()
    t2.start()
    t1.join()
    t2.join()
    end=time()
    print(f'total time is {end-start}')
main()

Begin to download Python从入门到住院.pdf
Begin to download how to pass a interview.pdf
how to pass a interview.pdf download completed,has consumed 7 seconds
Python从入门到住院.pdf download completed,has consumed 8 seconds
total time is 8.003484725952148


In [ ]:
# 线程锁
